In [4]:
from datetime import date
from blog.models import Post
from organizer.models import Tag, Startup, NewsLink

# Basic interaction with Django models

In [6]:
edut = Tag(name="Education", slug="education")

In [7]:
edut

<Tag: Education>

In [9]:
edut.name

'Education'

In [11]:
edut.save() #saves the data model into the database

In [12]:
edut.delete() #deletes data model from database

(1, {'blog.Post_tags': 0, 'organizer.Startup_tags': 0, 'organizer.Tag': 1})

In [13]:
edut #still in memory

<Tag: Education>